# MiddleWare: Human In The Loop

In [42]:
from langchain.agents import create_agent
from langchain.agents.middleware import HumanInTheLoopMiddleware 
from langgraph.checkpoint.memory import InMemorySaver 

agent = create_agent(
    model="openai:gpt-4o",
    tools=[write_file_tool, execute_sql_tool, read_data_tool],
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "write_file": True,  # All actions (accept, edit, respond) allowed
                "execute_sql": {"allow_accept": True, "allow_respond": True},  # No editing allowed
                # Safe operation, no approval needed
                "read_data": False,
            },
            # Prefix for interrupt messages - combined with tool name and args to form the full message
            # e.g., "Tool execution pending approval: execute_sql with query='DELETE FROM...'"
            # Individual tools can override this by specifying a "description" in their interrupt config
            description_prefix="Tool execution pending approval",
        ),
    ],
    # Human-in-the-loop requires checkpointing to handle interrupts.
    # In production, use a persistent checkpointer like AsyncPostgresSaver.
    checkpointer=InMemorySaver(),  
)

NameError: name 'write_file_tool' is not defined

## Setup

In [43]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [44]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("openai:gpt-5")

In [45]:
from typing import TypedDict

# Setup your agent runtime context
class Context(TypedDict):
    first: str
    last: str
    

In [55]:
import re

from langgraph.runtime import Runtime, get_runtime
from langchain_core.tools import tool

def _execute_sql(query: str) -> str:
    """Execute a SQLite command and return results.
    Named parameters like :first and :last will be filled from runtime context."""
    runtime = get_runtime(Context)
    try:
        placeholders = re.findall(r":(\w+)", query)

        if placeholders:
            params = {
                p: runtime.context[p]
                for p in placeholders
                if p in runtime.context
            }
            return db.run(query, parameters=params)
        else:
            return db.run(query)
    except Exception as e:
        return f"Error: {e}"
@tool
def execute_nonemployee_sql(query: str) -> str:
    """Execute a SQLite command and return results.
    Call this for any access that do not access the employee table.
    Named parameters like :first and :last will be filled from runtime context."""
    return _execute_sql(query)

@tool
def execute_employee_sql(query: str) -> str:
    """Execute a SQLite command that access the employee table and return results.
    Named parameters like :first and :last will be filled from runtime context."""
    return _execute_sql(query)



In [73]:
SYSTEM = """You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
- If a query involves a specific customer, always use the named parameters
  :first and :last (instead of hard-coding values).
  Example: SELECT InvoiceId, Total FROM Invoice
           JOIN Customer ON Invoice.CustomerId = Customer.CustomerId
           WHERE Customer.FirstName = :first AND Customer.LastName = :last;
  Assume the customer first and last name are known and will be inserted into the query later.
"""

In [96]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage
from langgraph.checkpoint.memory import InMemorySaver

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_employee_sql, execute_nonemployee_sql],
    prompt=SYSTEM,
    checkpointer=InMemorySaver(),
    context_schema=Context,
    middleware=[
        HumanInTheLoopMiddleware( 
            interrupt_on={
                "execute_employee_sql": {"allow_accept": True, "allow_respond": True},  # No editing allowed
            },
            description_prefix="Tool execution pending approval",
        ),
    ],

)



In [99]:
question = "What was the total on my last invoice?"

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    config = {"configurable": {"thread_id": "1"}} 
)
print(result["messages"][-1].content)

Your most recent invoice total is 5.94. Invoice ID 374, dated 2013-07-04.


In [97]:
from langgraph.types import Command
question = "What are the names of all the employees?"

config = {"configurable": {"thread_id": "1"}} 

result = agent.invoke(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    config=config
)
if '__interrupt__' in result:
    print(f"\033[1;3;31m{80*'-'}\033[0m")
    print(f"\033[1;3;31m Interrupt:{result['__interrupt__'][-1].value[-1]['description']}\033[0m")

    result = agent.invoke(
        Command( 
            resume=[{"type": "accept"}]  # or "edit", "respond"
        ), 
        config=config # Same thread ID to resume the paused conversation
    )
    print(f"\033[1;3;31m{80*'-'}\033[0m")
           
print(result["messages"][-1].content)

--------------------------------------------------------------------------------
 Interrupt:Tool execution pending approval

Tool: execute_employee_sql
Args: {'query': 'SELECT FirstName, LastName FROM Employee ORDER BY LastName, FirstName;'}
--------------------------------------------------------------------------------
- Andrew Adams
- Laura Callahan
- Nancy Edwards
- Steve Johnson
- Robert King
- Michael Mitchell
- Margaret Park
- Jane Peacock


In [82]:
result.keys()

dict_keys(['messages'])

In [68]:
config

Available objects for config:
    AliasManager
    DisplayFormatter
    HistoryManager
    IPCompleter
    IPKernelApp
    LoggingMagics
    MagicsManager
    OSMagics
    PrefilterManager
    ScriptMagics
    StoreMagics
    ZMQInteractiveShell


In [10]:
question = "What were the titles?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================

Could you clarify which titles you mean?
- Employee job titles (from the Employee table)
- Album titles
- Track titles (song names)
- Something else (e.g., playlist names)

If it’s for a specific customer’s purchases, please provide the customer’s first and last name and any time/filter (e.g., last invoice, all time).


## Add memory

In [11]:
from langgraph.checkpoint.memory import InMemorySaver

In [12]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql, convert_currency],
    prompt=SystemMessage(content=SYSTEM),
    context_schema=Context,
    checkpointer = InMemorySaver(),
)

In [15]:
question = "What was the total on my last invoice?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What was the total on my last invoice?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_Hn8akHWS0gzz3LezPzhBZgzo)
 Call ID: call_Hn8akHWS0gzz3LezPzhBZgzo
  Args:
    query: SELECT i.InvoiceId, i.InvoiceDate, i.Total
FROM Invoice AS i
JOIN Customer AS c ON i.CustomerId = c.CustomerId
WHERE c.FirstName = :first AND c.LastName = :last
ORDER BY i.InvoiceDate DESC, i.InvoiceId DESC
LIMIT 1;
================================= Tool Message =================================
Name: execute_sql

[(374, '2013-07-04 00:00:00', 5.94)]
================================== Ai Message ==================================

$5.94


In [16]:
question = "What were the titles?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    {"configurable": {"thread_id": "1"}},
    context= {"first": "Frank", "last": "Harris"},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What were the titles?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_C8WZtNsUGv00p7YFgsQQDtL8)
 Call ID: call_C8WZtNsUGv00p7YFgsQQDtL8
  Args:
    query: SELECT t.Name AS TrackTitle
FROM InvoiceLine AS il
JOIN Track AS t ON il.TrackId = t.TrackId
WHERE il.InvoiceId = (
  SELECT i.InvoiceId
  FROM Invoice AS i
  JOIN Customer AS c ON i.CustomerId = c.CustomerId
  WHERE c.FirstName = :first AND c.LastName = :last
  ORDER BY i.InvoiceDate DESC, i.InvoiceId DESC
  LIMIT 1
)
ORDER BY il.InvoiceLineId ASC
LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

[('Holier Than Thou',), ('Through The Never',), ('My Friend Of Misery',), ('The Wait',), ('Blitzkrieg',)]
================================== Ai Message ==================================

- Holier Than Thou
- Through The Never
- My Friend